In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier


from transformers import PreTrainedTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


from datasets import Dataset
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import sys
import gc

In [ ]:
# Different from online submission that use absolute path, we here use relative path for convenience

test = pd.read_csv('llm-detect-ai-generated-text/test_essays.csv')
sub = pd.read_csv('llm-detect-ai-generated-text/sample_submission.csv')
train = pd.read_csv("daigt-v2-train-dataset/train_v2_drcat_02.csv", sep=',')

In [ ]:
train = train.drop_duplicates(subset=['text'])
train.reset_index(drop=True, inplace=True)
train.head()

In [ ]:
excluded_prompt_name_list = ['Distance learning','Grades for extracurricular activities','Summer projects']
train = train[~(train['prompt_name'].isin(excluded_prompt_name_list))]
train = train.drop_duplicates(subset=['text'])
train.reset_index(drop=True, inplace=True)

In [ ]:
test.text.values

In [ ]:
LOWERCASE = False
VOCAB_SIZE = 305220

In [ ]:
# Creating Byte-Pair Encoding tokenizer
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))


# Adding normalization and pre_tokenizer
raw_tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFC()] + [normalizers.Lowercase()] 
    if LOWERCASE else []
)


raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

# Adding special tokens and creating trainer instance
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(
    vocab_size=VOCAB_SIZE, 
    special_tokens=special_tokens
)


# Creating huggingface dataset object
dataset = Dataset.from_pandas(test[['text']])

def train_corp_iter():
    """
    A generator function for iterating over a dataset in chunks.
    """    
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

# Training from iterator REMEMBER it's training on test set...
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)

tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token  = "[UNK]",
    pad_token  = "[PAD]",
    cls_token  = "[CLS]",
    sep_token  = "[SEP]",
    mask_token = "[MASK]",
)



# Tokenize test set with new tokenizer
tokenized_texts_test = []
for text in tqdm(test['text'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))


# Tokenize train set
tokenized_texts_train = []
for text in tqdm(train['text'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

In [ ]:
print(tokenized_texts_test[1])
print()
print(tokenized_texts_test[2])

In [ ]:
def dummy(text):
    """
    A dummy function to use as tokenizer for TfidfVectorizer. 
    It returns the text as it is since we already tokenized it.
    """
    return text



# Fitting TfidfVectoizer on test set
vectorizer = TfidfVectorizer(
    ngram_range   = (3, 5), 
    lowercase     = False, 
    sublinear_tf  = True, 
    analyzer      = 'word',
    tokenizer     = dummy,
    preprocessor  = dummy,
    token_pattern = None, 
    strip_accents ='unicode')


vectorizer.fit(tokenized_texts_test)

# Getting vocab
vocab = vectorizer.vocabulary_
print(vocab)


# Here we fit our vectorizer on train set but this time we use vocabulary from test fit.
vectorizer = TfidfVectorizer(
    ngram_range    = (3, 5), 
    lowercase      = False, 
    sublinear_tf   = True, 
    vocabulary     = vocab,
    analyzer       = 'word',
    tokenizer      = dummy,
    preprocessor   = dummy,
    token_pattern  = None, 
    strip_accents  ='unicode')

tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)
del vectorizer
gc.collect()

In [ ]:
y_train = train['label'].values

In [ ]:
tf_train

In [ ]:
tf_train.shape

In [ ]:
tf_test.shape

In [ ]:
def get_model():
    from catboost import CatBoostClassifier
    from sklearn.linear_model import LogisticRegression

    clf = MultinomialNB(alpha=0.0225)

    sgd_model = SGDClassifier(
        max_iter=9000, 
        tol=1e-4, 
        loss="modified_huber", 
        random_state=6743
    )

    p6 = {
        'n_iter': 3000, 
        'verbose': -1, 
        'objective': 'cross_entropy', 
        'metric': 'auc',
        'learning_rate': 0.00581909898961407, 
        'colsample_bytree': 0.78, 
        'colsample_bynode': 0.8
    }
    p6["random_state"] = 6743
    lgb = LGBMClassifier(**p6)

    # CatBoost classifier with specified iterations, learning rate, and subsample
    cat = CatBoostClassifier(
        iterations=3000, 
        verbose=0, 
        random_seed=6543,
        learning_rate=0.005599066836106983, 
        subsample=0.35,
        allow_const_label=True, 
        loss_function='CrossEntropy'
    )

    # Weights for the Voting Classifier, specifying the importance of each base classifier
    weights = [0.1, 0.31, 0.28, 0.67]

    # Create a Voting Classifier with the specified base classifiers and weights
    ensemble = VotingClassifier(
        estimators=[
            ('mnb', clf),
            ('sgd', sgd_model),
            ('lgb', lgb),
            ('cat', cat)
        ],
        weights=weights, 
        voting='soft', 
        n_jobs=-1
    )
    return ensemble

# Get the ensemble model
model = get_model()
print(model)

In [ ]:
# Check if the number of test samples is less than or equal to 5
if len(test.text.values) <= 5:
    # If true, generate a sample submission file
    sub.to_csv('submission.csv', index=False)
else:
    # If false, fit the ensemble model on the training data
    model.fit(tf_train, y_train)

    # Predict probabilities for the test data
    final_preds = model.predict_proba(tf_test)[:, 1]

    # Add the generated probabilities to the submission DataFrame
    sub['generated'] = final_preds
    
    print(sub['generated'].max(), sub['generated'].min(),sub['generated'].mean())
    
    # Save the submission file without index column
    sub.to_csv('submission.csv', index=False)